# GitHub Actions (CI/CD)

## Goals
- Understand what **GitHub Actions** is and how it enables CI/CD inside GitHub.
- Know **why** teams use it (tight GitHub integration, managed runners, ecosystem).
- Learn the key primitives: **events**, **workflows**, **jobs**, **steps**, and **runners**.
- See a realistic **workflow YAML pseudo-code** example.


## Prerequisites
- Familiarity with GitHub repositories, branches, and PRs.
- Basic CI concepts: build/test/artifacts.
- Helpful: Docker and your deployment target (cloud/Kubernetes).


## What is GitHub Actions?

**GitHub Actions** is GitHub's native automation platform for CI/CD and general workflows.

You define automation in **workflow YAML** files stored in your repository:
- Path: `.github/workflows/<name>.yml`
- Triggers: GitHub events (push, pull_request, release), schedules, or manual dispatch.
- Execution: one or more **jobs** running on **runners** (GitHub-hosted or self-hosted).


## What is it used for (and why teams choose it)?

Common uses:
- CI: build, unit/integration tests, linting, formatting checks.
- Security checks: dependency scanning, SAST, secret scanning (often integrated with GitHub).
- CD: deploy on merge/tag with environment approvals.
- Automation: issue/PR triage, release notes, scheduled maintenance tasks.

Why teams use it:
- **First-class GitHub integration** (PR checks, statuses, code owners, environments).
- **Managed runners** available instantly; easy scaling for typical workloads.
- Large ecosystem of reusable actions + marketplace.

Why teams avoid it:
- If you are not on GitHub (or have strict network/hosting constraints), it may not fit.
- Self-hosted runners require careful isolation and security hardening.


## How it works (mental model)

A useful picture:
```
GitHub event (push/PR/tag) -> Workflow -> Jobs -> Steps -> Runner
                                  |            |
                                  |            +--> run commands / call actions
                                  +--> parallelization + dependencies (needs)
```

Key mechanics:
- Each job runs in a **fresh VM/container** on GitHub-hosted runners (ephemeral by default).
- Jobs can run in parallel; `needs:` expresses dependencies.
- Steps are either shell commands (`run:`) or reusable components (`uses:`).
- Secrets are injected from GitHub (repo/org/environment secrets).
- Permissions to GitHub APIs can be controlled via `permissions:` and the `GITHUB_TOKEN`.


## Core concepts (minimum you should know)
- **Workflow**: a YAML file that defines automation.
- **Event trigger**: when the workflow runs (`on: push`, `on: pull_request`, `workflow_dispatch`).
- **Job**: a set of steps executed on the same runner.
- **Step**: a single action or command.
- **Runner**: the machine that executes jobs (GitHub-hosted vs self-hosted).
- **Artifacts**: files uploaded from a workflow for later download/use.
- **Environments**: deployment targets with protection rules (approvals, required reviewers).
- **Caching**: reuse dependencies/build outputs to speed up runs.


## Typical patterns
- **PR checks**: run tests/lint on `pull_request` and block merge on failures.
- **Build once, deploy many**: build an artifact/image once, then deploy/promote to environments.
- **Matrix testing**: test across versions/OSes using `strategy.matrix`.
- **OIDC to cloud**: prefer short-lived credentials (OpenID Connect) instead of long-lived secrets.


## Example: workflow YAML (pseudo-code)

Below is intentionally simplified but representative.

```yaml
# PSEUDO-CODE (.github/workflows/ci_cd.yml)
name: ci-cd

on:
  pull_request:
  push:
    branches: [main]
  workflow_dispatch:

permissions:
  contents: read
  packages: write
  id-token: write   # for OIDC to cloud (if used)

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with: { node-version: '20' }
      - run: npm ci
      - run: npm test -- --ci
      - uses: actions/upload-artifact@v4
        with:
          name: test-reports
          path: reports/**

  build_and_push_image:
    if: github.ref == 'refs/heads/main'
    needs: [test]
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: docker/login-action@v3
        with:
          registry: ghcr.io
          username: ${{ github.actor }}
          password: ${{ secrets.GITHUB_TOKEN }}
      - uses: docker/build-push-action@v6
        with:
          context: .
          push: true
          tags: ghcr.io/org/app:${{ github.sha }}

  deploy_staging:
    if: github.ref == 'refs/heads/main'
    needs: [build_and_push_image]
    runs-on: ubuntu-latest
    environment: staging
    steps:
      - run: ./deploy.sh staging ghcr.io/org/app:${{ github.sha }}

  deploy_prod:
    if: github.ref == 'refs/heads/main'
    needs: [deploy_staging]
    runs-on: ubuntu-latest
    environment: production  # can require approvals in GitHub settings
    steps:
      - run: ./deploy.sh prod ghcr.io/org/app:${{ github.sha }}
```


## What GitHub Actions is capable of
- CI/CD tightly integrated with GitHub PRs, checks, reviews, and environments.
- Fast onboarding: workflows live in-repo and run on managed runners.
- Strong reuse story: composite actions, reusable workflows, marketplace actions.

Good fit when:
- Your code lives on GitHub and you want minimal additional CI/CD infrastructure.

Less ideal when:
- You need highly customized build networks or strict hosting constraints that conflict with hosted runners.


## Pitfalls & quick tips
- Scope permissions: set `permissions:` explicitly and avoid broad defaults.
- Protect deployments with **environments** (approvals, required reviewers).
- Prefer OIDC for cloud auth instead of storing long-lived cloud keys in secrets.
- Treat self-hosted runners as sensitive (they can be targeted via workflow execution).
- Avoid leaking secrets in logs; never `echo` secrets.

## Exercises
- Add a matrix job testing 2 language/runtime versions.
- Split CI (PR) vs CD (main/tag) behavior using conditions.
- Add an environment-protected production deploy.

## References
- GitHub Actions Docs: https://docs.github.com/actions
- Workflow syntax: https://docs.github.com/actions/using-workflows/workflow-syntax-for-github-actions
- Security hardening: https://docs.github.com/actions/security-guides/security-hardening-for-github-actions
